임베딩이 잘 된 상황에서는 단어 벡터들의 평균만으로 텍스트 분류를 수행할 수 있음을 보고, 워드 임베딩의 중요성을 이해해보자.

영화 사이트 IMDB 영화 리뷰 데이터는 리뷰 텍스트에 리뷰가 긍정인 경우 1을, 부정인 경우 0으로 레이블링한 데이터로 25,000개의 훈련 데이터와 테스트 데이터 25,000개로 구성된 데이터이다.

## 1. 데이터 로드와 전처리

In [2]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.datasets import imdb

In [10]:
vocab_size = 20000

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)
print('훈련용 리뷰 개수 :',len(x_train))
print('테스트용 리뷰 개수 :',len(x_test))

훈련용 리뷰 개수 : 25000
테스트용 리뷰 개수 : 25000


In [6]:
# 이 데이터는 이미 정수 인코딩까지의 전처리가 진행되어져 있다.
print(x_train[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [7]:
print(y_train[0])

1


In [9]:
print('훈련용 리뷰의 평규 길이: {}'.format(np.mean(list(map(len, x_train)), dtype=int)))
print('테스트용 리뷰의 평균 길이: {}'.format(np.mean(list(map(len, x_test)), dtype=int)))

훈련용 리뷰의 평규 길이: 238
테스트용 리뷰의 평균 길이: 230


In [11]:
max_len = 400

x_train = pad_sequences(x_train, maxlen=max_len)
x_test = pad_sequences(x_test, maxlen=max_len)
print('x_train의 크기(shape) :', x_train.shape)
print('x_test의 크기(shape) :', x_test.shape)

x_train의 크기(shape) : (25000, 400)
x_test의 크기(shape) : (25000, 400)


## 2. 모델 설계하기
임베딩 벡터를 평균으로 사용하는 모델을 설계

* GlobalAveragePooling1D()은 입력으로 들어오는 모든 벡터들의 평균을 구하는 역할

In [12]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=max_len))
model.add(GlobalAveragePooling1D()) # 모든 단어 벡터의 평균 구하기
model.add(Dense(1, activation='sigmoid')) # 이진 분류 수행

In [13]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('embedding_average_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [14]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(x_train, y_train, batch_size=32, epochs=10, callbacks=[es, mc], validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 18s 27ms/step - loss: 0.6713 - acc: 0.6457 - val_loss: 0.5354 - val_acc: 0.8138

Epoch 00001: val_acc improved from -inf to 0.81380, saving model to embedding_average_model.h5
Epoch 2/10
625/625 [==============================] - 17s 28ms/step - loss: 0.4854 - acc: 0.8385 - val_loss: 0.3949 - val_acc: 0.8606

Epoch 00002: val_acc improved from 0.81380 to 0.86060, saving model to embedding_average_model.h5
Epoch 3/10
625/625 [==============================] - 18s 29ms/step - loss: 0.3518 - acc: 0.8843 - val_loss: 0.3389 - val_acc: 0.8718

Epoch 00003: val_acc improved from 0.86060 to 0.87180, saving model to embedding_average_model.h5
Epoch 4/10
625/625 [==============================] - 19s 30ms/step - loss: 0.2839 - acc: 0.9015 - val_loss: 0.3056 - val_acc: 0.8822

Epoch 00004: val_acc improved from 0.87180 to 0.88220, saving model to embedding_average_model.h5
Epoch 5/10
625/625 [==============================] - 18s 29ms/step - l

=> **별 다른 신경망을 추가하지 않고도 단어 벡터의 평균만으로도 높은 정확도를 얻어낸다.**